In [1]:
import pandas as pd

# Define file paths
file_path_return = "Data For Part A/10_Industry_Portfolios.CSV"
file_path_factor = "Data For Part A/F-F_Factors.csv"

# Load returns file with explicit header
df_rets = pd.read_csv(file_path_return, header=0, na_values=[-99.99, -999])

# Ensure first column is named 'Date'
df_rets.rename(columns={df_rets.columns[0]: 'Date'}, inplace=True)

# Convert 'Date' column to datetime format
df_rets['Date'] = pd.to_datetime(df_rets['Date'], format='%Y%m', errors='coerce')

# Convert numeric columns to appropriate data type
numeric_cols = df_rets.columns.drop('Date')
df_rets[numeric_cols] = df_rets[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Set Date as index and convert to PeriodIndex
df_rets.set_index('Date', inplace=True)
df_rets.index = df_rets.index.to_period('M')

# Load factor file with explicit header
df_factor = pd.read_csv(file_path_factor, header=0, na_values=[-99.99, -999])

# Ensure first column is named 'Date'
df_factor.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)

# Convert 'Date' column to datetime format
df_factor['Date'] = pd.to_datetime(df_factor['Date'], format='%Y%m', errors='coerce')

# Convert numeric columns to appropriate data type
numeric_cols = df_factor.columns.drop('Date')
df_factor[numeric_cols] = df_factor[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Set Date as index and convert to PeriodIndex
df_factor.set_index('Date', inplace=True)
df_factor.index = df_factor.index.to_period('M')

# Merge tables on index with left join
df_rets = df_rets.merge(df_factor[['RF']], left_index=True, right_index=True, how='left')


In [2]:
df_rets

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,RF
Date,,,,,,,,,,,
1926-07,1.45,15.55,4.69,-1.18,2.90,0.83,0.11,1.77,7.04,2.13,0.22
1926-08,3.97,3.68,2.81,3.47,2.66,2.17,-0.71,4.25,-1.69,4.35,0.25
1926-09,1.14,4.80,1.15,-3.39,-0.38,2.41,0.21,0.69,2.04,0.29,0.23
1926-10,-1.24,-8.23,-3.63,-0.78,-4.58,-0.11,-2.29,-0.57,-2.63,-2.84,0.32
1926-11,5.20,-0.19,4.10,0.01,4.71,1.63,6.43,5.42,3.71,2.11,0.31
...,...,...,...,...,...,...,...,...,...,...,...
2024-08,4.11,-5.16,1.79,-2.92,1.35,1.21,1.35,6.44,4.55,3.03,0.48
2024-09,0.99,14.24,2.83,-3.17,2.96,5.63,3.67,-1.81,5.35,0.66,0.40
2024-10,-2.59,-3.14,-3.60,-0.54,-0.21,2.36,-1.14,-3.11,0.71,1.11,0.39


In [3]:
# PART A
# 1--MAX SHARPE RATIO
# region
import numpy as np
import pandas as pd

# Compute excess returns
excess_returns = df_rets.iloc[:, :-1].sub(df_rets["RF"], axis=0)  # Subtract risk-free rate


# Function to compute max Sharpe portfolio weights
def max_sharpe_weights(returns):
    mean_ret = returns.mean()
    cov_matrix = returns.cov()
    inv_cov = np.linalg.inv(cov_matrix)

    # Compute tangency portfolio weights (unconstrained Markowitz solution)
    ones = np.ones(len(mean_ret))
    w_tangency = inv_cov @ mean_ret / (ones @ inv_cov @ mean_ret)

    return w_tangency


# Rolling window (5 years = 60 months)
window = 60
weights_list = []

for i in range(window - 1, len(excess_returns)):
    rolling_data = excess_returns.iloc[i - window + 1 : i + 1]  # Select 60 months of data
    weights = max_sharpe_weights(rolling_data)  # Compute optimal weights
    weights_list.append(weights)

# Convert list to DataFrame
dates = excess_returns.index[window-1:]
weights_df1 = pd.DataFrame(weights_list, index=dates, columns=excess_returns.columns)

weights_df1
# endregion

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1931-06,3.845585,-0.101518,-0.932185,-0.492752,2.007715,1.827132,-2.999195,2.572849,1.067613,-5.795244
1931-07,3.298068,-0.382247,-0.962862,-0.208554,1.772496,1.648591,-2.207438,2.223008,0.544382,-4.725444
1931-08,3.740837,-0.546504,-1.037348,-0.124926,2.310590,2.174561,-2.577138,2.714610,0.801903,-6.456585
1931-09,-5.800873,1.796977,0.977875,0.832529,-7.150685,-4.685237,6.724094,-6.580073,-1.832066,16.717459
1931-10,-6.290331,1.388535,1.476940,0.697121,-7.658258,-4.834227,6.970734,-6.815507,-2.082982,18.147973
...,...,...,...,...,...,...,...,...,...,...
2024-08,-0.711588,-0.250610,-0.883209,0.837761,4.360208,-4.531295,-2.240551,1.480012,0.462699,2.476573
2024-09,-0.711538,-0.081821,-0.898638,0.670868,3.704659,-3.408833,-1.722556,1.054507,0.687999,1.705352
2024-10,-0.871192,-0.196314,-1.689171,0.791987,4.253682,-3.673980,-1.536804,0.490914,1.033716,2.397163


In [4]:
# Validate the table to make sure the sum of weights are 1
row_sums = weights_df1.sum(axis=1)
row_sums.sum(axis=0)

1123.000000000001

In [5]:
# 2--MAX SHARPE RATIO, SHORT-SALE CONSTRAINED
# region
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Compute excess returns
excess_returns = df_rets.iloc[:, :-1].sub(df_rets["RF"], axis=0)  # Subtract risk-free rate


# Function to compute negative Sharpe ratio (for minimization)
def neg_sharpe(weights, mean_ret, cov_matrix, rf):
    port_return = np.dot(weights, mean_ret)  # Expected portfolio return
    port_vol = np.sqrt(weights @ cov_matrix @ weights.T)  # Portfolio standard deviation
    sharpe_ratio = (port_return - rf) / port_vol
    return -sharpe_ratio  # Negative Sharpe to minimize


# Rolling window (5 years = 60 months)
window = 60
weights_list = []

for i in range(window - 1, len(excess_returns)):
    rolling_data = excess_returns.iloc[i - window + 1 : i + 1]  # Select 60 months of data
    mean_ret = rolling_data.mean()
    cov_matrix = rolling_data.cov()
    rf = df_rets["RF"].iloc[i]  # Risk-free rate of current month

    # Initial guess (equal weights)
    num_assets = len(mean_ret)
    init_guess = np.ones(num_assets) / num_assets

    # Define bounds (no short-selling)
    bounds = [(0, 1) for _ in range(num_assets)]  # Weights between 0 and 1

    # Constraint: Weights sum to 1
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

    # Optimize using SLSQP (local search)
    result = minimize(neg_sharpe, init_guess, args=(mean_ret, cov_matrix, rf),
                      method='SLSQP', bounds=bounds, constraints=constraints)

    weights_list.append(result.x)  # Store optimized weights

# Convert list to DataFrame
dates = excess_returns.index[window - 1:]
weights_df2 = pd.DataFrame(weights_list, index=dates, columns=excess_returns.columns)

weights_df2
# endregion

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1931-06,1.221942e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,3.455377e-01,0.000000e+00,9.433408e-02,5.601282e-01,0.000000e+00
1931-07,3.402015e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2.619740e-01,0.000000e+00,4.318963e-01,3.061297e-01,0.000000e+00
1931-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2.928811e-01,0.000000e+00,3.285021e-01,3.786168e-01,1.218557e-17
1931-09,1.725494e-17,3.766518e-17,1.642640e-16,4.943355e-17,0.000000,7.059381e-02,5.130735e-17,0.000000e+00,9.294062e-01,7.465184e-17
1931-10,3.138740e-17,1.682575e-16,1.977046e-16,0.000000e+00,0.000000,1.804112e-16,1.115156e-16,0.000000e+00,1.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...
2024-08,5.512322e-17,1.066353e-02,0.000000e+00,1.019158e-01,0.887421,9.879950e-18,0.000000e+00,5.869568e-18,2.535066e-17,0.000000e+00
2024-09,8.026957e-17,3.623753e-02,2.659333e-17,6.417172e-02,0.899591,6.224040e-17,0.000000e+00,0.000000e+00,2.884554e-17,0.000000e+00
2024-10,0.000000e+00,1.088039e-02,0.000000e+00,8.467451e-02,0.904445,9.817744e-17,0.000000e+00,5.316593e-17,3.586161e-17,0.000000e+00


In [6]:
row_sums = weights_df2.sum(axis=1)
row_sums.sum(axis=0)

1123.0000000000007

In [7]:
# 3--ASSET INVERSELY RELATED TO VARIANCE
# region
import numpy as np
import pandas as pd

# Compute excess returns (subtract risk-free rate)
excess_returns = df_rets.iloc[:, :-1].sub(df_rets["RF"], axis=0)

# Rolling window (5 years = 60 months)
window = 60
weights_list = []

for i in range(window - 1, len(excess_returns)):
    rolling_data = excess_returns.iloc[i - window + 1 : i + 1]  # Select past 60 months

    # Compute variances of each asset
    variances = rolling_data.var()

    # Compute inverse variance weights
    inv_var_weights = 1 / variances
    inv_var_weights /= inv_var_weights.sum()  # Normalize to sum to 1

    weights_list.append(inv_var_weights.values)  # Store weights

# Convert list to DataFrame
dates = excess_returns.index[window - 1:]
weights_df3 = pd.DataFrame(weights_list, index=dates, columns=excess_returns.columns)

weights_df3
# endregion

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1931-06,0.205547,0.048718,0.072432,0.082189,0.046031,0.145273,0.085403,0.134451,0.060391,0.119565
1931-07,0.205281,0.051157,0.071455,0.082899,0.045543,0.144356,0.086288,0.135800,0.060839,0.116381
1931-08,0.206855,0.051120,0.071516,0.080777,0.045481,0.144366,0.086048,0.136377,0.060819,0.116640
1931-09,0.186822,0.055023,0.076457,0.085270,0.049867,0.151017,0.088149,0.124189,0.063658,0.119548
1931-10,0.186358,0.055352,0.077622,0.082827,0.050360,0.153314,0.085994,0.122063,0.064128,0.121983
...,...,...,...,...,...,...,...,...,...,...
2024-08,0.176199,0.016939,0.098989,0.029369,0.085507,0.103227,0.106366,0.163569,0.130512,0.089325
2024-09,0.176921,0.016814,0.099541,0.029357,0.085751,0.102145,0.106434,0.163327,0.130131,0.089579
2024-10,0.175723,0.016843,0.098798,0.029495,0.085918,0.102514,0.106430,0.163262,0.130985,0.090032


In [8]:
row_sums = weights_df3.sum(axis=1)
row_sums.sum(axis=0)

1123.0

In [9]:
#4--WEIGHTS INVERSELY RELATED TO VOLATILITY
# region
import numpy as np
import pandas as pd

# Compute excess returns (subtract risk-free rate)
excess_returns = df_rets.iloc[:, :-1].sub(df_rets["RF"], axis=0)

# Rolling window (5 years = 60 months)
window = 60
weights_list = []

for i in range(window - 1, len(excess_returns)):
    rolling_data = excess_returns.iloc[i - window + 1 : i + 1]  # Select past 60 months

    # Compute standard deviations of each asset
    volatilities = rolling_data.std()

    # Compute inverse volatility weights
    inv_vol_weights = 1 / volatilities
    inv_vol_weights /= inv_vol_weights.sum()  # Normalize to sum to 1

    weights_list.append(inv_vol_weights.values)  # Store weights

# Convert list to DataFrame
dates = excess_returns.index[window - 1:]
weights_df4 = pd.DataFrame(weights_list, index=dates, columns=excess_returns.columns)

weights_df4
# endregion

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1931-06,0.147388,0.071755,0.087493,0.093199,0.069748,0.123908,0.095004,0.119203,0.079890,0.112411
1931-07,0.147204,0.073485,0.086849,0.093545,0.069336,0.123442,0.095438,0.119728,0.080137,0.110837
1931-08,0.147833,0.073491,0.086924,0.092381,0.069319,0.123501,0.095347,0.120035,0.080160,0.111010
1931-09,0.139672,0.075800,0.089352,0.094361,0.072161,0.125576,0.095941,0.113877,0.081530,0.111729
1931-10,0.139505,0.076030,0.090035,0.093004,0.072520,0.126534,0.094766,0.112904,0.081835,0.112867
...,...,...,...,...,...,...,...,...,...,...
2024-08,0.138064,0.042808,0.103484,0.056367,0.096179,0.105676,0.107271,0.133024,0.118824,0.098303
2024-09,0.138363,0.042654,0.103784,0.056362,0.096327,0.105133,0.107318,0.132941,0.118664,0.098454
2024-10,0.137866,0.042682,0.103375,0.056483,0.096401,0.105301,0.107293,0.132887,0.119029,0.098682


In [10]:
row_sums = weights_df4.sum(axis=1)
row_sums.sum(axis=0)

1123.0

In [11]:
#5--ASSETS HAVE SAME WEIGHT
# All assets 10%.

weights_df5 = df_rets.iloc[:, :-1]
weights_df5 = weights_df5.loc["1931-06":]
weights_df5[:] = 0.1

weights_df5

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1931-06,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1931-07,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1931-08,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1931-09,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1931-10,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...
2024-08,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2024-09,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2024-10,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [12]:
row_sums = weights_df5.sum(axis=1)
row_sums.sum(axis=0)

1122.9999999999998

In [13]:
#6--WEIGHTS LINEARLY RELATED TO MARKET CAP

import pandas as pd
import numpy as np

# Load number of firms dataset
df_firms = pd.read_csv("Data For Part A/10_Industry_Portfolios_Firms.csv", na_values=[-99.99, -999])

# Load average market capitalization dataset
df_size = pd.read_csv("Data For Part A/10_Industry_Portfolios_MktCap.csv", na_values=[-99.99, -999])

# Rename first column to 'Date'
df_firms.rename(columns={df_firms.columns[0]: 'Date'}, inplace=True)
df_size.rename(columns={df_size.columns[0]: 'Date'}, inplace=True)

# Convert 'Date' column to datetime format
df_firms['Date'] = pd.to_datetime(df_firms['Date'], format='%Y%m', errors='coerce')
df_size['Date'] = pd.to_datetime(df_size['Date'], format='%Y%m', errors='coerce')

# Convert numerical columns to float type
numeric_cols_firms = df_firms.columns.drop('Date')
numeric_cols_size = df_size.columns.drop('Date')

df_firms[numeric_cols_firms] = df_firms[numeric_cols_firms].apply(pd.to_numeric, errors='coerce')
df_size[numeric_cols_size] = df_size[numeric_cols_size].apply(pd.to_numeric, errors='coerce')

# Set Date as index and convert to PeriodIndex (monthly data)
df_firms.set_index('Date', inplace=True)
df_size.set_index('Date', inplace=True)

df_firms.index = df_firms.index.to_period('M')
df_size.index = df_size.index.to_period('M')

# Step 1: Compute Market Capitalization
df_market_cap = df_firms * df_size  # Multiply # of firms by avg market cap

# Step 2: Shift Market Capitalization by One Month to Use Prior Month Data
df_market_cap_shifted = df_market_cap.shift(1)  # Shift market cap one month back

# Step 3: Compute Market Cap Weights Using Prior Month Data
weights_df6 = df_market_cap_shifted.div(df_market_cap_shifted.sum(axis=1), axis=0)

# Step 4: Drop Rows Before June 1931
weights_df6 = weights_df6.loc["1931-06":]

# Display the modified weights
weights_df6

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1931-06,0.121070,0.065037,0.202307,0.080498,0.069731,0.082857,0.054004,0.007964,0.151530,0.165002
1931-07,0.124140,0.062097,0.195085,0.080643,0.070268,0.087357,0.060359,0.008185,0.155149,0.156717
1931-08,0.119118,0.059708,0.195258,0.087205,0.069435,0.091362,0.055095,0.009196,0.147828,0.165795
1931-09,0.122594,0.062641,0.187281,0.090078,0.066707,0.093269,0.058063,0.010120,0.149775,0.159471
1931-10,0.123331,0.060910,0.189251,0.099261,0.070272,0.095559,0.060529,0.010222,0.135490,0.155176
...,...,...,...,...,...,...,...,...,...,...
2024-08,0.029892,0.018487,0.072010,0.032666,0.393789,0.019200,0.106390,0.088790,0.025649,0.213126
2024-09,0.030810,0.020301,0.073247,0.032797,0.380850,0.019501,0.105252,0.087779,0.026828,0.222634
2024-10,0.031431,0.018867,0.073039,0.031000,0.378371,0.019207,0.104509,0.091509,0.027382,0.224685


In [14]:
row_sums = weights_df6.sum(axis=1)
row_sums.sum(axis=0)

1123.0

In [15]:
# 7--MINIMUM VARIANCE PF
# region
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Step 1: Use raw asset returns (not excess returns)
asset_returns = df_rets.iloc[:, :-1]  # Drop the risk-free rate column

# Step 2: Initialize list to store weights
min_variance_weights = []

# Function to calculate portfolio variance
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# Step 3: Set window size
window = 60

# Step 4: Loop over rolling windows
for i in range(window - 1, len(asset_returns)):
    rolling_window = asset_returns.iloc[i - window + 1 : i + 1]
    cov_matrix = rolling_window.cov()

    num_assets = cov_matrix.shape[0]
    initial_weights = np.ones(num_assets) / num_assets

    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = [(0, 1) for _ in range(num_assets)]  # Long-only constraint

    result = minimize(portfolio_variance, initial_weights, args=(cov_matrix,), bounds=bounds, constraints=constraints)

    min_variance_weights.append(result.x)

# Step 5: Build DataFrame
dates = asset_returns.index[window - 1:]
weights_df7 = pd.DataFrame(min_variance_weights, index=dates, columns=asset_returns.columns)

# endregion

In [16]:
row_sums = weights_df7.sum(axis=1)
row_sums.sum(axis=0)

1123.0000000166597

In [17]:
#FINAL--PORTFOLIO COMPARISON
#region
df_rets_decimal = df_rets.astype(float) / 100

def compute_portfolio_returns(weight_dfs, df_rets_decimal):
    portfolio_returns_dict = {}

    for name, weights_df in weight_dfs.items():
        portfolio_ret = (df_rets_decimal.shift(-1) * weights_df).sum(axis=1).dropna()
        portfolio_ret.index = portfolio_ret.index.shift(1)  
        portfolio_returns_dict[name] = portfolio_ret

    df_portfolio_returns = pd.DataFrame(portfolio_returns_dict)
    return df_portfolio_returns

weight_dfs = {
    'Approach 1': weights_df1,
    'Approach 2': weights_df2,
    'Approach 3': weights_df3,
    'Approach 4': weights_df4,
    'Approach 5': weights_df5,
    'Approach 6': weights_df6,
    'Approach 7': weights_df7
}

df_portfolio_returns = compute_portfolio_returns(weight_dfs, df_rets_decimal)
first_valid_index = (df_portfolio_returns != 0).any(axis=1).idxmax()
df_portfolio_returns = df_portfolio_returns.loc[first_valid_index:]


pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", 1000)  # Set max width
pd.set_option("display.float_format", "{:.5f}".format)  # Format float precision

df_portfolio_returns

import numpy as np

# Define time periods
periods = {
    "July 1931 - Dec 2025": ("1931-07", "2025-12"),
    "Jan 1990 - Dec 2025": ("1990-01", "2025-12"),
    "Jan 2000 - Dec 2025": ("2000-01", "2025-12"),
}

# Define risk-free rate (e.g., assumed 3% annually, adjusted for monthly returns)
import pandas as pd

# Compute metrics
results = []
for period, (start, end) in periods.items():
    df_period = df_portfolio_returns.loc[start:end]
    risk_free_period = df_rets_decimal.loc[start:end].iloc[:, -1]  # Get actual risk-free rates for the period

    avg_return = df_period.mean()
    total_return = (1 + df_period).prod() - 1
    std_dev = df_period.std()
    sharpe_ratio = (avg_return - risk_free_period.mean()) / std_dev  # Adjusted Sharpe ratio

    for portfolio in df_period.columns:
        results.append({
            "Period": period,
            "Portfolio": portfolio,
            "Average Return": avg_return[portfolio],
            "Total Return": total_return[portfolio],
            "Sharpe Ratio": sharpe_ratio[portfolio]
        })

# Convert to DataFrame
df_results = pd.DataFrame(results)

# Pivot for better readability
df_results_pivot = df_results.pivot(index="Period", columns="Portfolio", values=["Average Return", "Total Return", "Sharpe Ratio"])

# Display results
df_results_pivot

# endregion


Average Return                                                                   Total Return                                                                        Sharpe Ratio                                                                  
Portfolio                Approach 1 Approach 2 Approach 3 Approach 4 Approach 5 Approach 6 Approach 7   Approach 1  Approach 2  Approach 3  Approach 4  Approach 5  Approach 6 Approach 7   Approach 1 Approach 2 Approach 3 Approach 4 Approach 5 Approach 6 Approach 7
Period                                                                                                                                                                                                                                                                  
Jan 1990 - Dec 2025         0.06328    0.00968    0.00918    0.00939    0.00967    0.00953    0.00782   -130.19409    34.85379    33.57501    35.87845    38.72514    35.01456   19.63634      0.04080    0.15682    0.18372    0.18173    0.17818    0.16645    0.16209
Jan 2000 - Dec 2025         0.07840    0.00779    0.00785    0.00795    0.00811    0.00756    0.00691    -37.37356     6.33267     7.34490     7.40647     7.51141     6.04795    5.56196      0.04353    0.13200    0.16250    0.15753    0.15119    0.13338    0.15220
July 1931 - Dec 2025        0.07795    0.01012    0.00977    0.00997    0.01017    0.00979    0.00883 -17023.31955 15379.49160 16783.18180 18899.44384 20797.18522 12616.13783 8172.33709      0.02812    0.13530    0.15236    0.15004    0.14720    0.13631    0.15554